In [1]:
import os
import re
import numpy as np
import pandas as pd

In [2]:
base_pheno = '/data1/guilimin/simons_vip/pheno/dataset.csv'
match_pheno = '/data1/guilimin/simons_vip/pheno/dataset_good_16p.csv'
raw_path = '/data1/guilimin/simons_vip/scores/scores_11_07/rmap_part/'
all_pheno = '/data1/guilimin/simons_vip/pheno/dataset_16p_all.csv'
all_s1 = '/data1/guilimin/simons_vip/pheno/dataset_16p_session1.csv'
all_s2 = '/data1/guilimin/simons_vip/pheno/dataset_16p_session2.csv'

In [3]:
base = pd.read_csv(base_pheno, delimiter=';')

In [4]:
match = pd.read_csv(match_pheno, delimiter=';')
match.rename(columns={'Unnamed: 0':'Subject'}, inplace=True)

In [5]:
base.ID = base.ID.str.replace(('.x'), ('x'))
base.rename(columns={'FD.':'FD', 'frames_OK.':'frames_OK'}, inplace=True)

In [6]:
merged = pd.merge(base, match, on='ID', how='right')

In [7]:
dropcols = [s for s in merged.columns if "_y" in s]
for colname in dropcols:
    merged.drop(colname, axis=1, inplace=True)

xcols = [s for s in merged.columns if "_x" in s]
rename_dict = dict()
for x in xcols:
    news = x.split('_x')[0]
    rename_dict[x] = news

merged.rename(columns=rename_dict, inplace=True)

In [8]:
# Check that the files exist
drop = list()
for ind, row in merged.iterrows():
    if row.session == 'session1':
        drop.append(ind)
        continue
    # File that we look for:
    fname = '{}_rmap_part.nii.gz'.format(row.IID.strip(' '))
    # Path that we are looking for
    fpath = os.path.join(raw_path, fname)
    # Does the file exist
    if not os.path.isfile(fpath):
        print('{} does not exist, drop em'.format(fname))
        drop.append(ind)

s14732xx6xFCAP1_session2_rest_rmap_part.nii.gz does not exist, drop em
s14786xx24xFCAP1_session2_rest_rmap_part.nii.gz does not exist, drop em
s14792xx10xFCAP1_session2_rest_rmap_part.nii.gz does not exist, drop em
s14806xx1xFCAP1_session2_rest_rmap_part.nii.gz does not exist, drop em
s14858xx4xFCAP1_session2_rest_rmap_part.nii.gz does not exist, drop em
s14861xx1xFCAP1_session2_rest_rmap_part.nii.gz does not exist, drop em
s14912xx1xFCAP1_session2_rest_rmap_part.nii.gz does not exist, drop em
s14930xx1xFCAP1_session2_rest_rmap_part.nii.gz does not exist, drop em
s14938xx1xFCAP1_session2_rest_rmap_part.nii.gz does not exist, drop em
s14951xx1xFCAP1_session2_rest_rmap_part.nii.gz does not exist, drop em
s14967xx25xFCAP1_session2_rest_rmap_part.nii.gz does not exist, drop em
s14976xx1xFCAP1_session2_rest_rmap_part.nii.gz does not exist, drop em
s14979xx2xFCAP1_session2_rest_rmap_part.nii.gz does not exist, drop em
s14985xx1xFCAP1_session2_rest_rmap_part.nii.gz does not exist, drop em
s14

In [9]:
# Correct some things in merged
# Relabel the genetic column to numeric
merged.genetic.replace({'deletion':1, 'non-familial0control':2, 'duplication':3, 'triplication': 3}, inplace=True)
# Genetic status: NaN -> control, triplication -> duplication
merged.genetic_status.replace({'triplication':'duplication'})
merged.genetic_status[pd.isnull(merged.genetic_status)] = 'control'
# Float columns with comma:
corr_com = ['FD', 'FD_scrubbed', 'bmi', 'hc']

for col in corr_com:
    merged[col] = merged[col].str.replace(',', '.').astype(np.float)

merged.rename(columns={'IID':''}, inplace=True)

/home/surchs/Venv/py35/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
cols = merged.columns.tolist()
new_cols = cols[1:] + cols[:1]
merged = merged[new_cols]

In [11]:
# Save merged
merged.to_csv(all_pheno, index=False)

In [12]:
# Split into sessions
s1 = merged[merged.session=='session1']
s2 = merged.drop(drop)

In [13]:
s1.to_csv(all_s1, index=False)
s2.to_csv(all_s2, index=False)

In [15]:
s1.genetic

0      1
2      1
4      3
6      3
8      1
10     1
12     3
14     3
15     3
17     1
19     3
21     3
23     3
25     1
27     3
29     3
31     1
33     1
35     1
37     1
39     1
41     2
43     2
45     2
47     2
49     3
51     3
53     3
55     2
57     1
      ..
162    2
164    2
165    2
167    2
169    3
171    3
173    1
175    1
177    2
179    2
181    2
183    2
185    2
187    2
189    2
191    2
193    2
194    2
196    2
198    2
200    2
201    2
203    2
205    2
207    2
209    2
211    2
213    2
215    1
217    2
Name: genetic, dtype: int64

In [16]:
s2.genetic

1      1
3      1
5      3
7      3
9      1
13     3
16     3
18     1
20     3
22     3
24     3
26     1
28     3
30     3
34     1
38     1
40     1
44     2
46     2
48     2
50     3
52     3
54     3
56     2
58     1
60     1
62     2
64     3
66     2
68     2
      ..
141    2
143    2
147    2
149    2
151    2
155    2
157    2
159    2
161    2
163    2
166    2
168    2
170    3
174    1
176    1
178    2
180    2
184    2
188    2
192    2
195    2
197    2
202    2
204    2
206    2
208    2
210    2
214    2
216    1
218    2
Name: genetic, dtype: int64